# IAQ and Survey Response Analysis
Exploring the relationship between the categorical survey responses and the IAQ measurements.

In [1]:
import warnings
warnings.filterwarnings('ignore')

## IAQ Distributions per Restfulness Score Reportings
Here we look at the distributions of IAQ measurements for each of the four ratings of restfulness.

In [2]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import statsmodels.api as sm

## Data Import
We will need just the morning survey results and the beacon data.

### Morning EMAs

In [72]:
ema = pd.read_csv('../data/processed/bpeace2-morning-survey.csv',index_col=0,parse_dates=True,infer_datetime_format=True)
ema.head()

,ID,Content,Stress,Lonely,Sad,Energy,TST,SOL,NAW,Restful
2020-07-31 14:25:00,hfttkth7,1,2,1.0,3,0,6.0,10.0,3.0,0
2020-08-20 03:49:00,hfttkth7,1,1,0.0,1,2,7.0,20.0,1.0,1
2020-08-23 15:58:00,hfttkth7,1,1,1.0,2,0,6.0,25.0,0.0,0
2020-07-17 14:52:00,hfttkth7,1,1,0.0,1,2,7.0,20.0,3.0,2
2020-08-12 17:32:00,hfttkth7,1,3,1.0,2,2,6.0,20.0,0.0,1


### Beacon IAQ

In [73]:
beacon = pd.read_csv('../data/processed/bpeace2-beacon.csv',index_col=0,parse_dates=True)
beacon.head()

,TVOC,eCO2,Lux,Visible,Infrared,NO2,T_NO2,RH_NO2,CO,T_CO,...,PM_N_10,PM_C_1,PM_C_2p5,PM_C_4,PM_C_10,z,Beacon,Beiwe,Fitbit,REDCap
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30,idbkjh8u,22,4
2020-06-11 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30,idbkjh8u,22,4
2020-06-11 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30,idbkjh8u,22,4
2020-06-11 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30,idbkjh8u,22,4
2020-06-11 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30,idbkjh8u,22,4


## Pre-Processing

### Getting IAQ measurements for sleep survey responses
We have to cut the beacon data down to nights that preceded when participants responded to surveys distributed that next morning. This process is slightly different from the one where we cut the beacon data down based on the Fitbit night recordings since there might be a few mornings when participants submitted a survey but did not wear their Fitbit to bed. However, even if there are a few of these instances we cannot summarize the preceding night's IAQ measurements because we don't know when the participant was asleep. Therefore, we **CAN** use the Fitbit-reduced IAQ data as a starting point.

In [74]:
evening_iaq = pd.read_csv('../data/processed/bpeace2-fitbit-beacon-evening-iaq.csv',
                          index_col=0,parse_dates=['Timestamp','start_time','end_time'],infer_datetime_format=True)
evening_iaq.head()

,TVOC,eCO2,Lux,Visible,Infrared,NO2,T_NO2,RH_NO2,CO,T_CO,...,PM_C_4,PM_C_10,z,Beacon,Beiwe,Fitbit,REDCap,start_time,end_time,beiwe
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-03 06:20:00,192.840000,405.36,0.0000,0.00,0.0,NaN,27.0,53.400000,-0.47992,NaN,...,16.400084,16.604594,0.162959,44,4i7679py,38,37,2020-07-03 06:20:00,2020-07-03 16:10:00,4i7679py
2020-07-03 06:25:00,178.866667,400.00,0.0000,0.00,0.0,NaN,27.0,52.833333,-0.30360,NaN,...,20.737945,21.006936,0.088227,44,4i7679py,38,37,2020-07-03 06:20:00,2020-07-03 16:10:00,4i7679py
2020-07-03 06:30:00,188.760000,401.40,0.9792,0.48,0.0,NaN,27.0,53.000000,-0.22436,NaN,...,16.946462,17.167753,0.114067,44,4i7679py,38,37,2020-07-03 06:20:00,2020-07-03 16:10:00,4i7679py
2020-07-03 06:35:00,193.840000,406.04,2.0400,1.00,0.0,NaN,27.0,53.000000,-0.26328,NaN,...,16.101073,16.309091,0.149835,44,4i7679py,38,37,2020-07-03 06:20:00,2020-07-03 16:10:00,4i7679py
2020-07-03 06:40:00,198.840000,409.44,2.0400,1.00,0.0,NaN,27.0,53.000000,-0.40936,NaN,...,19.142773,19.383442,0.163325,44,4i7679py,38,37,2020-07-03 06:20:00,2020-07-03 16:10:00,4i7679py


Now we compare the sleep survey submission date to the end time of the sleep period for each participant and remove the nights that don't line up

In [75]:
survey_iaq = pd.DataFrame()
for pt in evening_iaq['Beiwe'].unique():
    # getting pt-specific dfs
    evening_iaq_pt = evening_iaq[evening_iaq['Beiwe'] == pt]
    ema_pt = ema[ema['ID'] == pt]
    survey_dates = ema_pt.index.date
    survey_only_iaq = evening_iaq_pt[evening_iaq_pt['end_time'].dt.date.isin(survey_dates)]
    
    #survey_iaq = survey_iaq.append()

In [91]:
survey_only_iaq['Restful'] = 0
for d in survey_dates:
    print(d)
    print(survey_only_iaq.index.date)
    if survey_only_iaq.index.date == d:
        print(survey_only_iaq.index[d]['Restful'])
        #survey_only_iaq[d]['Restful'] = ema_pt[d]['Restful']

2020-07-27
[datetime.date(2020, 6, 2) datetime.date(2020, 6, 2)
 datetime.date(2020, 6, 3) ... datetime.date(2020, 8, 27)
 datetime.date(2020, 8, 27) datetime.date(2020, 8, 27)]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [76]:
ema_pt

,ID,Content,Stress,Lonely,Sad,Energy,TST,SOL,NAW,Restful
2020-07-27 17:50:00,zdpffrox,2,1,0.0,0,2,6.0,15.0,2.0,2
2020-06-03 14:20:00,zdpffrox,2,2,0.0,0,2,7.5,10.0,4.0,2
2020-08-16 17:06:00,zdpffrox,2,1,0.0,0,2,7.0,5.0,1.0,2
2020-07-01 20:56:00,zdpffrox,2,1,0.0,0,2,7.0,10.0,5.0,1
2020-07-06 15:15:00,zdpffrox,1,2,0.0,1,0,7.0,30.0,4.0,1
2020-08-12 18:04:00,zdpffrox,2,0,0.0,0,2,9.0,15.0,1.0,2
2020-06-26 15:34:00,zdpffrox,2,2,0.0,0,1,6.0,7.0,0.0,1
2020-08-19 19:14:00,zdpffrox,2,1,0.0,0,2,6.5,10.0,1.0,1
2020-06-21 18:44:00,zdpffrox,2,1,0.0,0,2,7.0,5.0,3.0,2
2020-07-23 00:46:00,zdpffrox,3,1,0.0,0,2,8.0,15.0,3.0,2


In [65]:
survey_iaq.head()

,TVOC,eCO2,Lux,Visible,Infrared,NO2,T_NO2,RH_NO2,CO,T_CO,...,PM_C_4,PM_C_10,z,Beacon,Beiwe,Fitbit,REDCap,start_time,end_time,beiwe
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-04 07:10:00,128.04,400.0,6.2832,196613.0,3.0,NaN,27.0,52.0,-0.91648,NaN,...,16.368654,16.581035,0.154553,44,4i7679py,38,37,2020-07-04 07:07:00,2020-07-04 16:10:00,4i7679py
2020-07-04 07:15:00,124.52,400.0,5.8752,196612.8,3.0,NaN,27.0,52.0,-0.81324,NaN,...,17.515252,17.739183,0.159092,44,4i7679py,38,37,2020-07-04 07:07:00,2020-07-04 16:10:00,4i7679py
2020-07-04 07:20:00,97.08,400.0,4.2432,196612.0,3.0,NaN,27.0,51.0,-0.71760,NaN,...,17.275952,17.501635,0.298510,44,4i7679py,38,37,2020-07-04 07:07:00,2020-07-04 16:10:00,4i7679py
2020-07-04 07:25:00,102.24,400.0,4.2432,196612.0,3.0,NaN,27.0,51.0,-0.73624,NaN,...,15.176323,15.371221,0.287511,44,4i7679py,38,37,2020-07-04 07:07:00,2020-07-04 16:10:00,4i7679py
2020-07-04 07:30:00,106.84,400.0,4.6512,196612.2,3.0,NaN,27.0,51.0,-0.74156,NaN,...,19.510833,19.767971,0.263813,44,4i7679py,38,37,2020-07-04 07:07:00,2020-07-04 16:10:00,4i7679py


Now we have the measurements for nights when participants submitted surveys and the final piece is to include the restfulness scores for those evenings.